In [ ]:
import os
import csv

samples = []
with open('.\data\driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

print("loaded lines..",len(samples))



loaded lines.. 16811


In [2]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)
print("Training Sample size....",len(train_samples))
print("Validation Sample size....",len(validation_samples))

Training Sample size.... 13448
Validation Sample size.... 3363


In [3]:
import cv2
import numpy as np
import sklearn
import random

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        random.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            correction = 0.2
            images = []
            angles = []
            for batch_sample in batch_samples:
                name = '.\data\IMG'+batch_sample[0].split('IMG')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
                images.append(cv2.flip(center_image,1))
                angles.append(center_angle*-1.0)
                
                name = '.\data\IMG'+batch_sample[1].split('IMG')[-1]
                left_image = cv2.imread(name)
                left_angle = float(batch_sample[3])+correction
                images.append(left_image)
                angles.append(left_angle)
                images.append(cv2.flip(left_image,1))
                angles.append(left_angle*-1.0)
                
                name = '.\data\IMG'+batch_sample[2].split('IMG')[-1]
                right_image = cv2.imread(name)
                right_angle = float(batch_sample[3])-correction
                images.append(right_image)
                angles.append(right_angle)
                images.append(cv2.flip(right_image,1))
                angles.append(right_angle*-1.0)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [4]:
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [ ]:
from keras.models import Sequential,Model
from keras.layers import Flatten,Dense,Lambda,Cropping2D,Dropout
from keras.layers import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda((lambda x:(x/255.0) - 0.5),input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(24,5,5,subsample=(2,2),activation='relu'))
#model.add(MaxPooling2D())
model.add(Convolution2D(36,5,5,subsample=(2,2),activation='relu'))
#model.add(MaxPooling2D())
model.add(Convolution2D(48,5,5,subsample=(2,2),activation='relu'))
#model.add(MaxPooling2D())
model.add(Convolution2D(64,3,3,activation='relu'))
#model.add(MaxPooling2D())
model.add(Convolution2D(64,3,3,activation='relu'))
#model.add(MaxPooling2D())
#model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse',optimizer='adam')
history_object=model.fit_generator(train_generator, samples_per_epoch=len(train_samples)*6, validation_data=validation_generator,nb_val_samples=len(validation_samples)*6, nb_epoch=10)
model.save('model.h6')

Using TensorFlow backend.


Epoch 1/10
80688/80688 [==============================] - 1384s - loss: 0.0280 - val_loss: 0.0254
Epoch 2/10
80688/80688 [==============================] - 1351s - loss: 0.0251 - val_loss: 0.0239
Epoch 3/10
80688/80688 [==============================] - 1385s - loss: 0.0238 - val_loss: 0.0234
Epoch 4/10
32448/80688 [===========>..................] - ETA: 696s - loss: 0.0227

In [ ]:
import matplotlib.pyplot as plt
### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()